In [3]:
import pandas as pd
import numpy as np
import mysql.connector


In [6]:
customers = pd.read_csv('C:/Users/samee/project code and ds/Projects/Emergence Data Analyst/customers.csv')
subscriptions = pd.read_csv('C:/Users/samee/project code and ds/Projects/Emergence Data Analyst/subscriptions.csv')
events = pd.read_csv('C:/Users/samee/project code and ds/Projects/Emergence Data Analyst/events.csv')


In [7]:
customers.head()
customers.info()
subscriptions.describe()
events['event_type'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    1000 non-null   object
 1   signup_date    1000 non-null   object
 2   segment        1000 non-null   object
 3   country        1000 non-null   object
 4   is_enterprise  1000 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 32.4+ KB


event_type
signup         1094
trial_start     676
activated       404
churned         237
Name: count, dtype: int64

In [9]:
customers.isnull().sum()
subscriptions.isnull().sum()
events.isnull().sum()


event_id       0
customer_id    0
event_type     0
event_date     0
source         0
dtype: int64

In [10]:
customers.duplicated(subset='customer_id').sum()
subscriptions.duplicated(subset='subscription_id').sum()


np.int64(0)

In [29]:
customers['signup_date'] = pd.to_datetime(customers['signup_date'], format='%d-%m-%Y', errors='coerce')
subscriptions['start_date'] = pd.to_datetime(subscriptions['start_date'], format='%d-%m-%Y', errors='coerce')
subscriptions['end_date'] = pd.to_datetime(subscriptions['end_date'], format='%d-%m-%Y', errors='coerce')


In [22]:
#negative and zero revenue
subscriptions[subscriptions['monthly_price'] <= 0]

,subscription_id,customer_id,start_date,end_date,monthly_price,status


In [23]:
#multiple active subscription

subscriptions[subscriptions['status'] == 'Active'] \
    .groupby('customer_id').size() \
    .reset_index(name='active_count') \
    .query('active_count > 1')

,customer_id,active_count


In [24]:
#check event without customer

events[~events['customer_id'].isin(customers['customer_id'])]


,event_id,customer_id,event_type,event_date,source


In [ ]:
#check for missing critical field

subscriptions[subscriptions['start_date'].isna()]


In [19]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="saas_analytics"
)

In [20]:
query = "SELECT COUNT(*) FROM customers"
db_customers = pd.read_sql(query, conn)

print("CSV:", len(customers))
print("DB :", db_customers.iloc[0,0])


CSV: 1000
DB : 1000


C:\Users\samee\AppData\Local\Temp\ipykernel_10004\4021901664.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  db_customers = pd.read_sql(query, conn)


In [31]:
#primary key uniqueness

customers['customer_id'].is_unique
subscriptions['subscription_id'].is_unique


True

In [32]:
#forein key integrity

subscriptions[~subscriptions['customer_id'].isin(customers['customer_id'])]


,subscription_id,customer_id,start_date,end_date,monthly_price,status


In [33]:
#empty orphans
events[~events['customer_id'].isin(customers['customer_id'])]


,event_id,customer_id,event_type,event_date,source
